In [17]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools
from jaqalpaq import run
from jaqalpaq import emulator
from jaqalpaq.run import run_jaqal_file, run_jaqal_string, run_jaqal_batch, run_jaqal_circuit, frontend
# from jaqalpaq.run import * (identical to from jaqalpaq import run aside from needing "run" before everything)
# from jaqalpaq.run import frontend (needed to switch between emulator and experiment)
from jaqalpaq.parser import parse_jaqal_string

## 3 qubits

### Measure crosstalk with rabi oscillations

In [18]:
SingleIonRabi = f"""

//Pulse Definitions Import Statement
from qscout.v1.std usepulses *

//Define let parameters
let alpha 0.1
let qtarget 1

//Select your register of qubits
register q[3]

//jaqal circuit
prepare_all
Rx q[qtarget] alpha
measure_all
"""

In [20]:
angles = list(np.linspace(0,40*np.pi, 100))

results = []

for t in np.arange(3):
    #override dictionary
    override_dict={ "alpha": angles,
               "qtarget": t,
                "__repeats__": 100}

    #Run the circuit with the parameters being overwritten
    res = run_jaqal_string(SingleIonRabi, overrides=override_dict)
    results.append(res)

## Slow single-mode MS

In [26]:
MSmode = f"""
// Import JaqalPaw Code.
from .VariableModeMS usepulses *
from qscout.v1.std usepulses *

// Declare variables.
let target1 1
let target2 3
let mode 2
let global_duration -1
let beta 0.1
let beta2 {np.pi/2.}

register q[3]

// Prepare = Sideband cool, then Pump to F=0
prepare_all

VariableModeMS q[target1] q[target2] mode global_duration
<R q[0] beta beta2 | R q[1] beta beta2 | R q[2] beta beta2> 

measure_all
"""

In [27]:
MSParity = f"""

//Pulse Definitions Import Statement
from qscout.v1.std usepulses *

//Define let parameters
let alpha {np.pi/2.}
let eta 0.0
let t1 0
let t2 1

let beta 0.1
let beta2 {np.pi/2.}

//Select your register of qubits
register q[3]

//jaqal circuit
prepare_all
MS q[t1] q[t2] eta alpha
<R q[0] beta beta2 | R q[1] beta beta2 | R q[2] beta beta2> 
measure_all
"""

In [25]:
angles = list(np.linspace(0,2*np.pi, 100))
t1 = 1
t2 = 3
results = []

#CommonMode
override_dict={ "target1": t1,
                   "target2": t2,
               "mode": 0,
               "beta": angles,
                "__repeats__": 100}

#Run the circuit with the parameters being overwritten
resCM = run_jaqal_string(MSmode, overrides=override_dict)

#BestMode
override_dict={ "target1": t1,
                   "target2": t2,
               "mode": 1,
               "beta":angles,
                "__repeats__": 100}

#Run the circuit with the parameters being overwritten
resBM = run_jaqal_string(MSmode, overrides=override_dict)

#QSCOUT standard MD
override_dict={"beta":angles,
               "t1": t1,
                   "t2":t2,
                "__repeats__": 100}
#Run the circuit with the parameters being overwritten
resQS = run_jaqal_string(MSParity, overrides=override_dict)


ModuleNotFoundError: No module named 'jaqalpaw'